In [1]:
import json
from src.utils import DataConfig, alter_data
from src.autoencoder import Autoencoder
from src.plots import *
from data.font import _font_2, symbols2    

with open('./config_denoising.json', 'r') as f:
    data_config = json.load(f)

c = DataConfig(data_config, _font_2)

MUTATE_PROB = 0.1
for i, prob in enumerate([round(0.1*n,2) for n in range(1,6)]):
    mutated_data, qty_mutated = alter_data(c.input_data, prob)
    mutated_data = np.array(list(mutated_data))
    print(c.input_data.shape)
    print(mutated_data.shape)

    autoencoder = Autoencoder(mutated_data, c.input_data, c.latent_space_size,
                            0.001, c.bias, 20000, 1,
                            c.min_error, 2, [20,10], 
                            c.output_activation, c.hidden_activation, c.beta,
                            c.optimizer_method, c.alpha, c.beta1, c.beta2,
                            c.epsilon)
    autoencoder.train()

    plot_letters(mutated_data, f"Conjunto de entrenamiento con {qty_mutated} caracteres mutados")

    predicted = []
    for x in mutated_data:
        p = autoencoder.predict(x)
        predicted.append(p)
    plot_letters(predicted, "Eliminacion del ruido")


    arr = []
    for j in range(len(mutated_data)):
        value = autoencoder.latent_space(mutated_data[j])
        arr.append(value)
        print("Latent space value: ", value, " for letter in index ", j)

    plot_latent_space(np.array(arr), symbols2)


(32, 35)
(32, 35)
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [ ]:
import json
from src.utils import DataConfig, alter_data
from src.autoencoder import Autoencoder
from src.plots import *
from data.font import _font_2    

with open('./config_denoising.json', 'r') as f:
    data_config = json.load(f)

c = DataConfig(data_config, _font_2)

MUTATE_PROB = 0.2

ARQUITECTURAS = [[15], [20,10], [20,15,10]]
NODES = [1,2,3]
ARQUITECTURAS_COMPLETA = ["35x15x2x15x35", "35x20x10x2x10x20x35", "35x20x15x10x2x10x15x20x35"]

mutated_data = alter_data(c.input_data, MUTATE_PROB)

arr_of_errors = []
arr_of_epochs = []
for i in range(len(ARQUITECTURAS)):
    autoencoder = Autoencoder(mutated_data, c.input_data, c.latent_space_size,
                            c.learning_rate, c.bias, c.epochs, 1,
                            c.min_error, NODES[i], ARQUITECTURAS[i],
                            c.output_activation, c.hidden_activation, c.beta,
                            c.optimizer_method, c.alpha, c.beta1, c.beta2,
                            c.epsilon)
    mse_errors, total_epochs = autoencoder.train()
    arr_of_errors.append(mse_errors)
    arr_of_epochs.append(total_epochs)

colors = ['blue', 'green', 'red', 'orange']

fig, ax = plt.subplots()  
for i, arquitectura in enumerate(ARQUITECTURAS):
    ax.plot(range(arr_of_epochs[i]), arr_of_errors[i], color=colors[i], label=f"{ARQUITECTURAS_COMPLETA[i]}")
ax.set_title('Evaluacion del error de DAE con distintas arquitecturas') 
ax.set_xlabel('Epocas') 
ax.set_ylabel('Error (MSE)')  
ax.legend(loc='best', bbox_to_anchor=(1.2, 1.0))  
plt.show()  